# PPO2 on Solo8 v2 Vanilla for Quadrupedal Standing w/ a Multiplicitive Reward
Try to get the solo to stand on 4 feet stabley

## Ensure that Tensorflow is using the GPU

In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


## Define Experiment Tags

In [2]:
TAGS = ['solov2vanilla', 'gpu', 'home_pos_mulitiplicitive', 'unnormalized_actions']

# Import required libraries

In [3]:
from gym_solo.envs import solo8v2vanilla
from gym_solo.core import obs
from gym_solo.core import rewards
from gym_solo.core import termination as terms

import gym
import gym_solo

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Parse CLI arguments and register w/ wandb

This experiment will be using the auto trainer to handle all of the hyperparmeter running

In [4]:
from auto_trainer import params
import auto_trainer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Give the robot a total of 10 seconds simulation time to learn how to stand.

In [5]:
episode_length = 2 / solo8v2vanilla.Solo8VanillaConfig.dt
episode_length

2000.0

Create a basic config

In [6]:
config = params.WandbParameters().parse()

config.episodes = 12500
config.episode_length = episode_length

config.target_torso_height = 0.33698 # Found experimentally

config.num_workers = 6
config.eval_frequency = 50
config.eval_episodes = 3
config.fps = 15

# Create a 3 second gif
config.eval_render_freq = int(config.episode_length / (3 * config.fps))

config

Namespace(algorithm='PPO2', episode_length=2000.0, episodes=12500, eval_episodes=3, eval_frequency=50, eval_render_freq=44, fps=15, num_workers=6, policy='MlpPolicy', target_torso_height=0.33698)

In [7]:
config, run = auto_trainer.get_synced_config(config, TAGS)
config

wandb: Currently logged in as: agupta231 (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


{'episodes': 12500, 'episode_length': 2000.0, 'policy': 'MlpPolicy', 'algorithm': 'PPO2', 'num_workers': 6, 'eval_episodes': 3, 'eval_frequency': 50, 'eval_render_freq': 44, 'fps': 15, 'target_torso_height': 0.33698}

Add the following inputs to the robot / environment:

**Observations**
- TorsoIMU
- Motor encoder current values

**Reward**
- How flat the torso is :$f$
- Minimize the amount of control in the joints: $c$
- Minimize the amount of torso movement: $m$
- Keeping the torso at a given height: $h$

We'll compose the "standing" reward to be $\frac{f + h}{2}$ as $f,h \in [0, 1]$. Then, the final reward becomes:

$$reward = \frac{f + h}{2} cm$$

Note that since $c,m \in [0, 1]$, this enforces that $reward \in [0, 1]$

**Termination Criteria**
- Terminate after $n$ timesteps

In [8]:
def make_env(length, quad_standing_height):
    def _init():
        env_config = solo8v2vanilla.Solo8VanillaConfig()
        env = gym.make('solo8vanilla-v0', config=env_config, 
                       normalize_actions=False)

        env.obs_factory.register_observation(obs.TorsoIMU(env.robot))
        env.obs_factory.register_observation(obs.MotorEncoder(env.robot))
        env.termination_factory.register_termination(terms.TimeBasedTermination(length))
        
        stand_reward = rewards.AdditiveReward()
        stand_reward.client = env.client
        
        stand_reward.add_term(0.5, rewards.FlatTorsoReward(env.robot))
        stand_reward.add_term(0.5, rewards.TorsoHeightReward(env.robot, quad_standing_height))
        
        home_pos_reward = rewards.MultiplicitiveReward(1, stand_reward,
                                                          rewards.SmallControlReward(env.robot),
                                                          rewards.HorizontalMoveSpeedReward(env.robot, 0))
        home_pos_reward.client = env.client
        
        env.reward_factory.register_reward(1, home_pos_reward)

        return env
    return _init

### Create the Envs
Import the desired vectorized env

In [9]:
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common.vec_env import VecNormalize

Create training & testing environments

In [10]:
train_env = SubprocVecEnv([make_env(config.episode_length, 
                                    config.target_torso_height) 
                           for _ in range(config.num_workers)])

test_env = make_env(config.episode_length, 
                    config.target_torso_height)()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecatedWarning: register_reward is deprecated. This will probaby be removed for AdditiveReward down the line. For best future-proofing pratices, create an AdditiveReward and register it with weight 1


## Learning
And we're off!

In [11]:
model, config, run = auto_trainer.train(train_env, test_env, config, TAGS, 
                                        log_freq=1000, full_logging=False, run=run)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




-------------------------------------
| approxkl           | 0.0045465445 |
| clipfrac           | 0.059570312  |
| explained_variance | -0.382       |
| fps                | 387          |
| n_updates          | 1            |
| policy_entropy     | 17.025557    |
| policy_loss        | -0.013191761 |
| serial_timesteps   | 128          |
| time_elapsed       | 0.00022      |
| total_timesteps    | 768          |
| value_loss         | 0.028260427  |
-------------------------------------



wandb: WARNING Step cannot be set when using syncing with tensorboard. Please log your step values as a metric such as 'global_step'


KeyboardInterrupt: 